#  Описание работы

## Задания
* Разделить смесь в окне (можно ЕМ алгоритм или второй метод; __прочитай в серой книжке__). Посчитать энтропию. Все для каждой компоненты магнитного поля в отдельности.
* Придумать, что делать с пропущенными отсчётами:
    1. [Теорема Котельникова](https://ru.wikipedia.org/wiki/%D0%A2%D0%B5%D0%BE%D1%80%D0%B5%D0%BC%D0%B0_%D0%9A%D0%BE%D1%82%D0%B5%D0%BB%D1%8C%D0%BD%D0%B8%D0%BA%D0%BE%D0%B2%D0%B0)
* Оптимизировать работу кода, задейстовав все возможные потоки процессора - [параллельное программирование](https://www.youtube.com/watch?v=fKl2JW_qrso&list=RDCMUCCezIgC97PvUuR4_gbFUs5g&start_radio=1&rv=fKl2JW_qrso&t=28)
* Сделать подписи осей абсцисс для графиков смесей так, чтобы в качестве отметок выступали интервалы времени, которое захватывает окно. В целом еще будут неудобства связанные с пропусками. Может, лучше просто отмечать номер окна, как и было раньше (но тогда неудобно сравнивать информацию, полученную для тех же параметров, но с другим размером окна).

1. Первая стадия 
    * Данные по одной координате разделить на смеси в окне, считая что имеется __6__ нормальных законов. Суммарно получается 18-ть дополнительных массивов: мат. ож-е, ср. кв. откл. и веса для каждого закона.
    * Визуализировать полученные массивы (придумать как). Можно через составляющие валатильности и тренд.
    * В каждом окне считается энтропия $H = -\sum_{i=1}^{6} p_i \cdot \log{p_i}$
    
2. Далее разбиваемся на группы: построение многомерного ЕМ и изучения всплесков, их вер-ти.

## Информация от физиков

### Письмо

Пересылаю запрошенный в конце прошедшей недели набор файлов за 2023 год, включающий:
именования столбцов регистрируемых параметров + трехмесячные файлы изображения временных разверток + листинги этих разверток. __Период опроса__ -- 1 минута. Предобработка по ограничению уровня и скорости вариации индукции магнитиного поля. В сумме -- около 2.000.000 выборок. Кажущиеся дефекты с октября 2023 года соответствуют процессу предфильтрации, через месяц они исчезнут. Формат содержит только актуальные данные, необходимые для реконструкции.

Для проверочных кодов будем использовать четыре __синтетических__ параметра: SYM_H, ASYM_H, SYM_D, ASYM_D.







## Замечания






* Один и тот же файл выдают за разный - `omni_min_def_20230401_20230630.lst` и `omni_min_def_20230101_20230331.lst`. Это данные за Январь-Март-2023 и Апрель-Июнь-2023. Судя по всему потеряна именно информация со второго периода, так как значения переменной 'Day'.
* В данных за октябрь полностью отсутсвуют синтетические параметры. Да и в принципе мало данных за этот месяц.
* EM-алгоритм выдает пропуски на графиках в связи с тем, что на каком-то номере итерации среднеквадрат-е откл-е $\sigma$ становится равным нулю. Что превращает веса с индексами соответсвующими $\sigma_k=0$ на следующей итерации в неопределённые значения `nan`

### Памятка


* Не забывай при существенном изменении кода создавать новые ветки и работать на них.
* https://numpy.org/devdocs/user/basics.types.html - number types.
* https://plotly.com/python/3d-charts/ - plotly 3D
* https://plotly.github.io/plotly.py-docs/generated/plotly.express.histogram.html - plotly histogram
* https://plotly.com/python/reference/layout/#layout-title - plotly fig layout
* https://coolors.co/092327-0b5351-00a9a5-4e8098-90c2e7 - генератор политры цвета.

# Загрузка данных

In [ ]:
import pandas as pd
import numpy as np
import os

import histplot as hp # Персональный модуль визуализации
import essentials as es # Персональный модуль основных алгоритмов


# Позволяет использовать измененные модули без перезагрузки ядра
%load_ext autoreload
%autoreload 2

# Позволяет выводит графики в ячейках
%matplotlib inline

# Путь к папке с данными
PATH_DATA = os.getcwd() + '/Data/'

# Указание типов для упрощения считывания данных из *.csv файлов
TYPES = dtypes = {
    'Year': int, # str на самом деле, просто с int удобнее работать
    'Day': int, # str
    'Hour': int, # str
    'Minute': int, # str
    'BX': float,
    'BY': float,
    'BZ': float,
    'Vx_Velocity': float,
    'Vy_Velocity': float,
    'Vz_Velocity': float,
    'SYM/D': int,
    'SYM/H': int,
    'ASY/D': int,
    'ASY/H': int }

# os.listdir(PATH_DATA) # выводит содержимое папки, лежащей по пути 'PATH_DATA'

# Считываени данных
# df1 = pd.read_csv(PATH_DATA + "january_march.csv", dtype=TYPES)
# df2 = pd.read_csv(PATH_DATA + "april_jun.csv", dtype=TYPES)
# df3 = pd.read_csv(PATH_DATA + "july_september.csv", dtype=TYPES)
# df4 = pd.read_csv(PATH_DATA + "october.csv")
# df = pd.read_csv(PATH_DATA + "FULL_DATA.csv")

data_clean = pd.read_csv(PATH_DATA + "dropna_january_march.csv", dtype=TYPES)

from converter import time_related_id
time_related_id(data_clean)

# Обработка периода Январь-Март

В связи упомянутым замечанием про отсутсвие данных с периода Февраль-Июнь и построенным графиком всех данных, анализ данных будет осуществляться на подвыборке Январь-Март

## Визуализация данных

### Временные ряды и их гистограммы

In [ ]:
data_clean

In [ ]:
#custom_title = "Исходные данные"
#hp.show_genral_info(df1['BX'], add_title=custom_title, add_xaxis=df1['ydhm_id'])
    
custom_title_drop = "Пропуски \"склеены\"."
custom_xaxis = data_clean['ydhm_id']
hp.show_genral_info(data_clean['BX'], add_title=custom_title_drop, add_xaxis=custom_xaxis)

Комментарии:
1. Склеенные данные не выглядят сплошными из-за равномерности сетки по времени. На самом деле в "дырках" отсчетов нет

### 3D гистограммы временных рядов

In [ ]:

#hist_3D = hist3D(data['BX'], window_size=4000, step=100)
clean_hist_3D = es.construct_hist3D(data_clean['BX'], window_size=4000, step=100)

#hp.movable3D_hist(hist_3D)
hp.movable3D_hist(clean_hist_3D)

clean_BX_3D_0_3000_30 = es.construct_hist3D(data_clean['BX'][:3000], step=30)
clean_BY_3D_0_3000_30 = es.construct_hist3D(data_clean['BY'][:3000], step=30)
clean_BZ_3D_0_3000_30 = es.construct_hist3D(data_clean['BZ'][:3000], step=30)

hp.movable3D_hist(clean_BX_3D_0_3000_30)
hp.movable3D_hist(clean_BY_3D_0_3000_30)
hp.movable3D_hist(clean_BZ_3D_0_3000_30)

# ЕМ-алгоритм

* __Е-этап__
    1. Calculate unnormalized responsibilities: 
    $$
    \normalsize{ \quad \tilde\rho_k^{[i]} = \pi_k \cdot \frac{1}{\sigma_k \sqrt{2\pi}} \cdot \exp{\left(-\frac{(x^{[i]} - \mu_k)^2}{2\sigma^2}\right)} \equiv
    \pi_k \cdot \frac{1}{\sigma_k} \cdot \varphi \left(\frac{x^{[i]} - \mu_k}{2\sigma} \right) }
    $$
    2. Normilize responsibilities: 
    $$
    \normalsize{ \quad \rho_k^{[i]} = \frac{\tilde\rho_k^{[i]}}{\sum_{k=0}^{M-1} \tilde\rho_k^{[i]}} }
    $$
    3. Calculate class responsibilities: 
    $$
    \normalsize{ \quad \gamma_k = \sum_{i=0}^{N-1} \rho_k^{[i]} }
    $$
* __М-этап__
    1. Update the class probabilities: 
    $$
    \normalsize{ \quad \pi_k = \frac{\quad \gamma_k}{N} }
    $$
    2. Update the math. expectations: 
    $$
    \normalsize{ \quad \mu_k = \frac{1}{\gamma_k} \cdot \sum_{i=0}^{N-1} \rho_k^{[i]}x^{[i]} }
    $$
    3. Update the standard deviations:
    $$
    \normalsize{ \quad \sigma_k = \sqrt{\frac{1}{\gamma_k} \cdot \sum_{i=0}^{N-1} \rho_k^{[i]}\left(x^{[i]} - \mu_k \right)^2} }
    $$

## Реализация алгоритма

### Код для проверки корректности работы реализованного ЕМ-алгоритма

## Применение к склеиным данным.

In [ ]:
random_seed = 42 # for reproducability
n_iterations = 42
n_classes = 6
clean_mix_dicts = []
for n_classes in range(1,11):
    class_probs, mus, sigmas = es.EM(data_clean['BX'].values,
                                     n_classes, 
                                     n_iterations,
                                     random_seed)
    param = {}
    param.update({"class_probs": class_probs,
                  "mus" : mus,
                  "sigmas" : sigmas})
    clean_mix_dicts.append(param)
    print(f"Вероятности классов: {class_probs}")
    print(f"Мат. ожидания: {mus}")
    print(f"Ср. кв. отклонение: {sigmas}\n")

In [ ]:
hp.static2D_mixture(clean_mix_dicts, data_clean['BX'], mode='subplots')

In [ ]:
hp.static2D_mixture(clean_mix_dicts, data_clean['BX'])

In [ ]:
# СТОП ТОЧКА
2=1

In [ ]:
WS = 6000
EPS = 0.01
STEP = 1

In [ ]:
BX_mixtures2 = es.mixture_param(data_clean['BX'], n_class=2, eps=0.01, rseed=
                            step=1, window_size=3000, xaxis_id=data_clean['ydhm_id'])

In [ ]:
BX_mixtures3 = es.mixture_param(data_clean['BX'], n_class=3, eps=0.01,
                            step=1, window_size=3000, xaxis_id=data_clean['ydhm_id'])

In [ ]:
BX_mixtures4 = es.mixture_param(data_clean['BX'], n_class=4, eps=0.01,
                            step=1, window_size=3000, xaxis_id=data_clean['ydhm_id'])

In [ ]:
BX_mixtures5 = es.mixture_param(data_clean['BX'], n_class=5, eps=0.01,
                            step=1, window_size=3000, xaxis_id=data_clean['ydhm_id'])

In [ ]:
BX_mixtures6 = es.mixture_param(data_clean['BX'], n_class=6, eps=0.01,
                            step=1, window_size=3000, xaxis_id=data_clean['ydhm_id'])

In [ ]:
BX_mixtures8 = es.mixture_param(data_clean['BX'], n_class=8, eps=0.01,
                            step=1, window_size=3000, xaxis_id=data_clean['ydhm_id'])

In [ ]:
plot2 = hp.construct_mixture_2Dplot(BX_mixtures2)
plot2
plot2.write_html("/home/oplora/Documents/VMK/Dissertation/Magfield/Images/mixtures/2 comp/2_compomemts.html")

In [ ]:
plot3 = hp.construct_mixture_2Dplot(BX_mixtures3)
plot3
plot4.write_html("/home/oplora/Documents/VMK/Dissertation/Magfield/Images/mixtures/3 comp/3_compomemts.html")

In [ ]:
plot4 = hp.construct_mixture_2Dplot(BX_mixtures4)
plot4
plot4.write_html("/home/oplora/Documents/VMK/Dissertation/Magfield/Images/mixtures/4 comp/4_compomemts.html")

In [ ]:
plot5 = hp.construct_mixture_2Dplot(BX_mixtures5)
plot5
plot5.write_html("/home/oplora/Documents/VMK/Dissertation/Magfield/Images/mixtures/5 comp/5_compomemts.html")

In [ ]:
plot6 = hp.construct_mixture_2Dplot(BX_mixtures6)
plot6
plot6.write_html("/home/oplora/Documents/VMK/Dissertation/Magfield/Images/mixtures/6 comp/6_compomemts.html")

In [ ]:
plot8 = hp.construct_mixture_2Dplot(BX_mixtures8)
plot8
plot8.write_html("/home/oplora/Documents/VMK/Dissertation/Magfield/Images/mixtures/8 comp/8_compomemts.html")

# Черновики

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

components = data_multicol.columns.levels[1] # число законов в смеси
params = data_multicol.columns.levels[0] # число параметров в кадом законе

for i, low in enumerate(lows):
    X = data_multicol.loc[:,(params[0], low)].values
    Y = data_multicol.loc[:,(params[1], low)].values
    Z = data_multicol.loc[:,(params[2], low)].values
    fig = px.line_3d(df, x="gdpPercap", y="pop", z="year", color='country')
    fig.add_trace(Scatter(x=X, 
                          y=Y,
                          name=f"Закон №{i+1}",
                          showlegend=legend,
                          mode='lines', line=dict(
                              color=lows_colors[i])),
                  row = row_ind + 1,
                  col = 1,
                  )


df = px.data.gapminder().query("continent=='Europe'")
fig = px.line_3d(df, x="gdpPercap", y="pop", z="year", color='country')
fig.show()

data_multicol.loc[:,(parameter, low)].values
# Выделение данных
x = hist3D['bins'].values
y = hist3D['wind_numb'].values
z = hist3D['hist_freq'].values

# Построение 3D поверхности
fig = go.Figure(data=[go.Surface(z=z, x=x, y=y)])

# Персонализация изолиний и проекции
custom_contours_z = dict(
                        show=True,
                        usecolormap=True,
                        highlightcolor="limegreen",
                        project_z=True)
fig.update_traces(contours_z = custom_contours_z)

# Персонализация осей
custom_scene = dict(
            xaxis = dict(
                title='Интервалы гист-ы',
                color='grey'),
            yaxis = dict(
                title='№ Окна',
                color='grey'),
            zaxis = dict(
                title = 'Приращения '+hist3D.attrs.get('data_name')+", нТ",
                color = 'grey'))

# Название графика
custom_title = f"Компонента: {hist3D.attrs.get('data_name')}, " \
        f"кол-во данных: {hist3D.attrs.get('data_length')}, " \
        f"размер окна: {hist3D.attrs.get('window_size')}, "\
        f"кол-во интервалов {hist3D.attrs.get('bin_size')}, " \
        f"длина шага: {hist3D.attrs.get('step_size')}."
# Персонализация графика
fig.update_layout(title=custom_title,
                scene=custom_scene,
                autosize=True,
                width=1200, height=600,
                margin=dict(l=65, r=50, b=65, t=90))
fig.show()
'''
!pip install dash
# Вывод графика в отдельном окне (для Jupyter в теории)
from dash import Dash, dcc, html
app = Dash()
app.layout = html.Div([
    dcc.Graph(figure=fig)
])

# Turn off reloader if inside Jupyter
app.run_server(debug=True, use_reloader=False)
'''

In [ ]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np

# Ваш DataFrame с мульти-индексными колонками
df = BX_mixtures3

# Подготовка данных для графика
# Получение значений math_exp, st_dev и индексов из DataFrame
math_exp_values = df['math_exp'].values
st_dev_values = df['st_dev'].values
indexes = df.index

# Создание графика
fig = go.Figure()

# Добавление линий для каждого набора значений low1, low2, low3
for i in range(3):
    fig.add_trace(go.Scatter3d(
        x=math_exp_values[i*3:(i+1)*3],
        y=st_dev_values[i*3:(i+1)*3],
        z=indexes,
        mode='lines',
        name=f'low{i+1}'
    ))

# Настройка макета графика
fig.update_layout(
    scene=dict(
        xaxis_title='math_exp',
        yaxis_title='st_dev',
        zaxis_title='Indexes'
    ),
    title='3D График',
    margin=dict(l=0, r=0, b=0, t=0)
)

# Отображение графика
fig.show()

In [ ]:
import pandas as pd
import plotly.express as px

# Создаем DataFrame с данными
data = {
    ('math_exp', 'low1'): [-1.4638484738930757, -0.3129638849381339, 0.4378035783464271, 0.7917185393928099],
    ('math_exp', 'low2'): [1.1182969315826252, 1.1338414909560688, 1.1768693855543684, 1.1967888986347366],
    ('math_exp', 'low3'): [0.4379802455951592, 0.4458119700724697, 0.508690200775006, 0.5327087084880078],
    ('st_dev', 'low1'): [2.2948276788197286, 2.4044792987542873, 2.362156575142729, 2.522171944208071],
    ('st_dev', 'low2'): [0.33366225797421145, 0.32748350767888623, 0.32054687601580995, 0.3191312224879689],
    ('st_dev', 'low3'): [0.2453108578017542, 0.25982242505793873, 0.25758288583583244, 0.24378210192894956],
    ('cl_prob', 'low1'): [0.9312875693636951, 0.915404822395992, 0.8359370418678999, 0.8576766609066121],
    ('cl_prob', 'low2'): [0.03335583813221263, 0.04513145504086317, 0.10447779219930656, 0.09535686433529078],
    ('cl_prob', 'low3'): [0.03535659250409139, 0.039463722563146446, 0.05958516593279373, 0.046966474758]
}

df = pd.DataFrame(data)

# Собираем мульти-индекс для колонок
df.columns = pd.MultiIndex.from_tuples(df.columns)

# Меняем уровни мульти-индекса колонок местами
df = df.swaplevel(axis=1)

# Рисуем 3D линейный график
fig = px.line_3d(df, x=('math_exp'), y=('st_dev'), z=df.index, color=df.columns.get_level_values(0))

# Отображаем график
fig.show()